In [1]:
import numpy as np
import rasterio
from rasterio.enums import Resampling


In [2]:
# Define file paths
aspect_file = r"D:\ForestFires\Elevation Data-20241206T115300Z-001\Elevation Data\Uttarakhand_Aspect2.tif"
slope_file = r"D:\ForestFires\Elevation Data-20241206T115300Z-001\Elevation Data\Uttarakhand_Slope2.tif"
output_aspect = r"D:\ForestFires\Elevation Data-20241206T115300Z-001\Elevation Data\Uttarakhand_Aspect_500m.tif"
output_slope = r"D:\ForestFires\Elevation Data-20241206T115300Z-001\Elevation Data\Uttarakhand_Slope_500m.tif"


In [3]:
# Resample function for Aspect
def resample_aspect(input_file, output_file, scale_factor):
    with rasterio.open(input_file) as src:
        aspect = src.read(1)

        # Convert aspect to sine and cosine
        aspect_rad = np.deg2rad(aspect)
        sin_aspect = np.sin(aspect_rad)
        cos_aspect = np.cos(aspect_rad)

        # Resample sine and cosine
        sin_resampled = src.read(
            1,
            out_shape=(
                int(src.height / scale_factor),
                int(src.width / scale_factor)
            ),
            resampling=Resampling.average
        )
        cos_resampled = src.read(
            1,
            out_shape=(
                int(src.height / scale_factor),
                int(src.width / scale_factor)
            ),
            resampling=Resampling.average
        )

        # Calculate aggregated aspect
        aggregated_aspect = np.arctan2(sin_resampled, cos_resampled)
        aggregated_aspect_deg = (np.rad2deg(aggregated_aspect) + 360) % 360

        # Save output
        transform = src.transform * src.transform.scale(scale_factor, scale_factor)
        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=aggregated_aspect_deg.shape[0],
            width=aggregated_aspect_deg.shape[1],
            count=1,
            dtype=rasterio.float32,
            crs=src.crs,
            transform=transform
        ) as dst:
            dst.write(aggregated_aspect_deg, 1)

In [4]:
# Resample function for Slope
def resample_slope(input_file, output_file, scale_factor):
    with rasterio.open(input_file) as src:
        slope_resampled = src.read(
            1,
            out_shape=(
                int(src.height / scale_factor),
                int(src.width / scale_factor)
            ),
            resampling=Resampling.average
        )

        transform = src.transform * src.transform.scale(scale_factor, scale_factor)
        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=slope_resampled.shape[0],
            width=slope_resampled.shape[1],
            count=1,
            dtype=slope_resampled.dtype,
            crs=src.crs,
            transform=transform
        ) as dst:
            dst.write(slope_resampled, 1)



In [5]:
# Execute resampling
scale_factor = 500 / 30  # Assuming input resolution is 30m
resample_aspect(aspect_file, output_aspect, scale_factor)
resample_slope(slope_file, output_slope, scale_factor)

print("Resampling completed!")

Resampling completed!


In [6]:
# Define file paths
input_file = r"C:\Users\CiaX\Downloads\uttarakhand_pd_2020_1km.tif"
output_file = r"C:\Users\CiaX\Downloads\uttarakhand_pd_2020_500m.tif"

In [7]:
# Resample function for population data
def resample_population(input_file, output_file, scale_factor):
    with rasterio.open(input_file) as src:
        # Calculate new shape
        new_height = int(src.height * scale_factor)
        new_width = int(src.width * scale_factor)

        # Read and resample data
        data = src.read(
            1,  # Reading the first band
            out_shape=(src.count, new_height, new_width),
            resampling=Resampling.bilinear  
        )

        # Adjust the transform to the new resolution
        transform = src.transform * src.transform.scale(1 / scale_factor, 1 / scale_factor)

        # Write the resampled data to a new file
        with rasterio.open(
            output_file,
            "w",
            driver="GTiff",
            height=new_height,
            width=new_width,
            count=1,
            dtype=data.dtype,
            crs=src.crs,
            transform=transform
        ) as dst:
            dst.write(data, 1)

In [8]:
# Define scale factor for upsampling
scale_factor = 2  # 1km to 500m requires doubling the resolution

# Perform resampling
resample_population(input_file, output_file, scale_factor)

print(f"Resampled population data saved to: {output_file}")

Resampled population data saved to: C:\Users\CiaX\Downloads\uttarakhand_pd_2020_500m.tif
